<a href="https://colab.research.google.com/github/VenkteshV/question_gen_PAWS_ADS/blob/master/Retrieval_based_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# from google.colab import drive

# drive.mount('/content/drive')

In [7]:
%cd /content/drive/MyDrive/question_gen_PAWS_ADS

/content/drive/MyDrive/question_gen_PAWS_ADS


In [8]:
!git pull

Already up to date.


In [9]:
!pip install scann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import scann
import pandas as pd

In [11]:
df = pd.read_csv('./article_data.csv')
df = df.dropna()

In [12]:
df

,topic,summary
0,inverted index,"In computer science, an inverted index (also r..."
1,Inverted index,"In computer science, an inverted index (also r..."
2,Search engine indexing,"Search engine indexing is the collecting, pars..."
3,Database index,A database index is a data structure that impr...
4,Boolean model of information retrieval,The (standard) Boolean model of information re...
...,...,...
604,Spectral centroid,The spectral centroid is a measure used in dig...
605,K-means clustering,k-means clustering is a method of vector quant...
606,Pappus's centroid theorem,"In mathematics, Pappus's centroid theorem (als..."
607,List of second moments of area,The following is a list of second moments of a...


In [13]:
!pip install transformers
!pip install tensorboard
!pip install sentence_transformers
import pandas as pd
import numpy as np
import torch
import pandas as pd
from time import time
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import tensorflow_hub as hub # For USE
from sentence_transformers import SentenceTransformer
import os
import tensorflow as tf
import datetime
from tensorboard.plugins import projector


def generate_embeddings(data_ls):
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = model.encode(data_ls)
    return embeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
count = 0
sentences = []
for i in df['summary']:
  if  type(i) != type("") or len(i) <= 0 :
    count+=1
  else:
    sentences.append(i.replace("\n",' '))

print(count, " sentences removed.")

0  sentences removed.


In [15]:
len(sentences)

600

In [16]:
t =time()
embeds = generate_embeddings(sentences)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Time taken:  50 s,  0 m


In [17]:
embeds.shape

(600, 384)

In [18]:
embeds_df = pd.DataFrame(embeds)

In [20]:
import json
questions = []
answers = []
with open('./nq-project/NQ-open.dev.jsonl') as f:

  for line in f.readlines():
    questions.append(json.loads(line)['question'])
    answers.append(json.loads(line)['answer'])

In [21]:
len(questions)

3610

In [22]:
t =time()
embeds_questions = generate_embeddings(questions)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

Time taken:  34 s,  0 m


In [23]:
embeds_questions.shape

(3610, 384)

In [24]:
embeds.shape

(600, 384)

In [25]:
embeds_df = pd.DataFrame(embeds)## articles dataset
embeds_questions_df = pd.DataFrame(embeds_questions) ## natural questions

In [26]:
!curl https://storage.cloud.google.com/natural_questions/v1.0-simplified/nq-dev-all.jsonl.gz


In [27]:
!tar -xvzf nq-dev-all.jsonl.gz

tar (child): nq-dev-all.jsonl.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [28]:
import numpy as np

k = int(np.sqrt(embeds_questions_df.shape[0]))

In [29]:

searcher = scann.scann_ops_pybind.builder(embeds_questions_df, 10, "dot_product").tree(num_leaves=k, num_leaves_to_search=int(k/20), training_sample_size=2500).score_brute_force(2).reorder(7).build()


In [30]:
indices = searcher.search(embeds_df.loc[0])
indices

(array([ 169, 2720,  277, 3236,  550, 3600, 1303, 2416, 2212,  528],
       dtype=uint32),
 array([11.870464 , 11.15177  ,  9.543509 ,  9.137827 ,  8.917534 ,
         7.856188 ,  7.8028293,  7.6533303,  7.580413 ,  7.251926 ],
       dtype=float32))

In [31]:
pd.DataFrame(questions).loc[indices[0]]

,0
169,what type of database is library literature an...
2720,what is the multiplication sign on the computer
277,where is the slide placed on the microscope
3236,where is south pole located on a map
550,what was icq as related to instant messaging
3600,oklahoma's 10 geographic regions are defined b...
1303,what is one element a topographic map shows
2416,what is a dropped pin on google maps for
2212,who invented the cornell method of note taking
528,where is fulda and what is its significance


In [32]:
questions[169]

'what type of database is library literature and information science'

In [33]:
pd.DataFrame(sentences).loc[0]

0    In computer science, an inverted index (also r...
Name: 0, dtype: object

In [34]:
indices = searcher.search(embeds_df.loc[3])
indices

(array([ 169,  484,  550,  528,    9,  597, 2295,  697, 2572,  396],
       dtype=uint32),
 array([20.10228  , 12.031965 , 10.229166 ,  7.741317 ,  7.5837975,
         7.380877 ,  7.0943813,  6.9982867,  6.9256487,  6.8818145],
       dtype=float32))

In [35]:
pd.DataFrame(questions).loc[indices[0]]

,0
169,what type of database is library literature an...
484,is a network connection device that can build ...
550,what was icq as related to instant messaging
528,where is fulda and what is its significance
9,what is the maximum data rate for the 802.11a ...
597,when did athens emerges as wealthiest greek ci...
2295,what is the ultimate objective of financial ma...
697,legislation regarding data protection and secu...
2572,what is the transfer speed for sata 1
396,what does istj mean in a personality test


In [74]:
list(np.where(df.topic=='edit distance',1,0)).index(1)

72

In [75]:
indices = searcher.search(embeds_df.loc[72])
indices

(array([1114,  893, 2863, 3604, 2212, 3333, 1151, 1320, 3113,  169],
       dtype=uint32),
 array([9.524683 , 7.8025823, 7.1778717, 7.1246376, 6.568034 , 6.512067 ,
        6.3251743, 6.147476 , 6.025811 , 5.8856707], dtype=float32))

In [77]:
#embeds_df[df.topic=='edit distance'].iloc[0]
embeds_df.iloc[72]

0     -0.093837
1     -0.139901
2     -0.055061
3     -0.101094
4      0.160006
         ...   
379    0.273782
380    0.057418
381    0.200561
382    0.096391
383    0.054940
Name: 72, Length: 384, dtype: float32

In [78]:
pd.DataFrame(questions).loc[indices[0]]

,0
1114,internet based test of english as a foreign la...
893,a synonym for the content component of communi...
2863,method used by a writer to develop a character
3604,natural resources can be divided into what two...
2212,who invented the cornell method of note taking
3333,who is often associated with printing the firs...
1151,what does a heart rate of 131 mean
1320,turkish finnish and hungarian belong to which ...
3113,the general term for software that is designed...
169,what type of database is library literature an...


In [79]:
from bs4 import BeautifulSoup

In [80]:
from time import time

In [ ]:

nq_sentences = []
with open('./nq-project/v1.0-simplified_nq-dev-all.jsonl','r') as f:
  for line in f.readlines():
    soup = BeautifulSoup(json.loads(line)['document_html'], 'html.parser')
    nq_sentences.append(soup.get_text())

In [ ]:
len(nq_sentences)

In [ ]:
t =time()
embeds_nq = generate_embeddings(sentences)
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

In [ ]:
embeds_nq_df = pd.DataFrame(embeds_nq)

In [ ]:

searcher = scann.scann_ops_pybind.builder(embeds_nq_df, 10, "dot_product").tree(num_leaves=k, num_leaves_to_search=int(k/20), training_sample_size=2500).score_brute_force(2).reorder(7).build()


In [ ]:
# indices = searcher.search(embeds_df[df.topic=='edit distance'].iloc[0])
indices = searcher.search(embeds_df[])
indices